# Collaborative filtering

- Coger subsets aleatorios
- Ver si usar item based collab porque tiene mas escalabilidad, user based necesita calcular la similitud de todos los pares de usuarios

## Data loading

In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
RANDOM_STATE = 5482

In [86]:
#df_users = pd.read_csv('data/train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'play_count'])
df_users_raw = pd.read_csv('data/Million Song Dataset kaggle/User Listening History.csv')

In [87]:
df_music = pd.read_csv('data/Million Song Dataset kaggle/Music Info.csv')

In [88]:
df_users_raw.dtypes

track_id     object
user_id      object
playcount     int64
dtype: object

In [89]:
df_users_raw.shape

(9711301, 3)

In [90]:
num_distinct_users = df_users_raw['user_id'].nunique()
num_distinct_users

962037

We only take into account users that have listened more than 5 songs

In [91]:
df_user_counts = df_users_raw['user_id'].value_counts().reset_index()
df_user_counts.columns = ['user_id', 'count']
df_user_counts

,user_id,count
0,ec6dfcf19485cb011e0b22637075037aae34cf26,784
1,4e11f45d732f4861772b2906f81a7d384552ad12,384
2,726da71c2c2ea119119a7957517fccd028d1be76,376
3,113255a012b2affeab62607563d03fbdf31b08e7,367
4,7adec7f006cb09482d36609d205293d8b61f030e,366
...,...,...
962032,19e70c61de7bbb0ca3fc39c81a7e6b7eede69aa1,1
962033,7154895e6b692062033eecb3fe59ad240bb02811,1
962034,5d563bc9f80ddb3bce489c84b453409a88bd37e9,1
962035,779074ba35bd5f14913635819233a01dc5ecc9de,1


In [92]:
df_users = df_users_raw[df_users_raw['user_id'].isin(df_user_counts[df_user_counts['count'] > 5]['user_id'])].reset_index(drop=True)
#df_users = df_users_raw
df_users.shape

(8332242, 3)

In [93]:
data_df = df_users_raw.groupby('track_id').size().reset_index(name='count')

In [94]:
data_df

,track_id,count
0,TRAAAED128E0783FAB,161
1,TRAAAHD128F42635A5,2
2,TRAAAQO12903CD8E1C,1
3,TRAABJS128F9325C99,945
4,TRAABXA128F9326351,18
...,...,...
30454,TRZZYMU128E0792400,1140
30455,TRZZZCL128F428BB80,19
30456,TRZZZCN128F9317A03,15
30457,TRZZZHL128F9329CFB,116


In [95]:
df_music.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [96]:
df_music_info = df_music[['track_id', 'name', 'artist', 'energy']]
df_music_info.head()

,track_id,name,artist,energy
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430


In [97]:
df_music_count = df_music_info.merge(
    df_users.groupby('track_id').size().reset_index(name='playcount'),
    on='track_id',
    how='left'
)
df_music_count = df_music_count.fillna(0)
print(df_music_count.shape)
df_music_count.head()

(50683, 5)


,track_id,name,artist,energy,playcount
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,0.918,32.0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,0.892,283.0
2,TROUVHL128F426C441,Come as You Are,Nirvana,0.826,0.0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0.664,17.0
4,TRLNZBD128F935E4D8,Creep,Radiohead,0.430,0.0


In [98]:
# df_music_count = df_music_info.merge(
#     df_users.groupby('track_id').size().reset_index(name='playcount'),
#     on='track_id',
#     how='inner'
# )
# print(df_music_count.shape)
# df_music_count.head()

## Interaction Matrix

In [99]:
df_users_agg = df_users.groupby('user_id')['playcount'].agg(
    #total_playcount='sum',
    max_playcount='max'
).reset_index()
df_users_agg = df_users_agg.rename(columns={'playcount': 'max_playcount'})

In [100]:
df_users_rating = df_users.merge(df_users_agg, on='user_id')

We create 2 interaction matrix
- Binary version: 1 indicates a user has interacted with the song and 0 indicates he has not
- Weighted version: Rating in range (0,1], calculated as (user playcount) / (playcount of user's most listened song). Gives more rating to those more listened by a user

In [101]:
df_users_rating['rating'] = df_users_rating['playcount'] / df_users_rating['max_playcount']
df_users_rating[:30]

,track_id,user_id,playcount,max_playcount,rating
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
5,TRGRDEC128F423C07D,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
6,TRRYCBO128F932A2C7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
7,TRUTULC128F4293712,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50
8,TRAAHSY128F147BB5C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,2,1.00
9,TRDLMWP128F426BF6C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,2,0.50


In [102]:
df_users_agg[df_users_agg['user_id'] == 'b80344d063b5ccb3212f76538f3d9e43d87dca9e']

,user_id,max_playcount
333783,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2


For the interaction matrix, we shall use a sparse matrix, due to users only listening to a small subset of the songs

In [103]:
from scipy.sparse import csr_matrix, csc_matrix

We cant reproduce next line because it needs 218 GB of RAM to create the table

In [104]:
#df_music_pivoted = df_users.pivot(index='user_id', columns='track_id', values='rating').fillna(0)

We codify each user and song to a unique integer ID

In [105]:
user_codes, user_uniques = pd.factorize(df_users['user_id'])
track_codes, track_uniques = pd.factorize(df_users['track_id'])

We create the interaction matrix as a sparse csr matrix for user-based collaborative filtering and as csc for item-based collaborative filtering

In [106]:
interaction_matrix_csr_binary = csr_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csr_weighted = csr_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [107]:
interaction_matrix_csc_binary = csc_matrix((np.ones(len(df_users)), (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))
interaction_matrix_csc_weighted = csc_matrix((df_users_rating['rating'], (user_codes, track_codes)),shape=(len(user_uniques), len(track_uniques)))

In [108]:
interaction_matrix_csr_binary.shape

(464573, 30201)

In [109]:
interaction_matrix_csc_weighted.shape

(464573, 30201)

In [110]:
# Calcular el espacio en memoria ocupado por interaction_matrix en MB
memory_usage_bytes = (
    interaction_matrix_csc_weighted.data.nbytes +
    interaction_matrix_csc_weighted.indptr.nbytes +
    interaction_matrix_csc_weighted.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de interaction_matrix: {memory_usage_mb:.2f} MB")

Espacio en memoria de interaction_matrix: 95.47 MB


The sparse matrix has a size of only 114.81 MB, while the normal matrix as a dataframe was 218 GB

In [111]:
interaction_matrix_csc_binary[:10, :30].todense()

matrix([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

In [112]:
interaction_matrix_csc_weighted[:10, :30].todense()

matrix([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 1.        , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.25      , 0.5       , 1.        , 0.25      , 0.25      ,
         0.5       , 0.75      , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ,

## Item based collaborative filtering

We use cosine similarity as a similarity metric. We can precalculate the similarities.

In [113]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csc_matrix

# Calculate item-based (track-based) cosine similarity
# Transpose interaction_matrix_weighted to get items as rows
item_based_similarity = cosine_similarity(interaction_matrix_csc_weighted.T, dense_output=False)
item_based_similarity = csc_matrix(item_based_similarity)

In [114]:
memory_usage_bytes = (
    item_based_similarity.data.nbytes +
    item_based_similarity.indptr.nbytes +
    item_based_similarity.indices.nbytes
)
memory_usage_mb = memory_usage_bytes / (1024 ** 2)
print(f"Espacio en memoria de item_based_similarity: {memory_usage_mb:.2f} MB")

Espacio en memoria de item_based_similarity: 934.39 MB


Solo 7 segundos de ejecucion

In [115]:
item_based_similarity.shape

(30201, 30201)

In [116]:
class ItemBasedRecommender:
    def __init__(self, interaction_matrix, item_similarity, track_uniques, df_music_info):
        self.interaction_matrix = interaction_matrix
        self.item_similarity = item_similarity
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index
        user_ratings = self.interaction_matrix[self.user_index]
        interacted = user_ratings.nonzero()[1]
        similarities = self.item_similarity.dot(user_ratings.T).toarray().ravel()
        similarities[interacted] = -np.inf
        top_n_index = np.argpartition(similarities, -n)[-n:]
        top_n_index = top_n_index[np.argsort(similarities[top_n_index])[::-1]]

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Similarity: {similarities[index]}")

        track_ids = self.track_uniques[top_n_index].tolist()
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()
        index_to_similarity = {idx: similarities[idx] for idx in top_n_index}

        self.recommendations = [(track_id, energy, index_to_similarity[self.track_uniques.tolist().index(track_id)], False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids_ordered = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

### Example

In [117]:
user_index = 0 #User for wich recommendations will be made

Songs the user has listened to

In [118]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [119]:
item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(user_index, n=20)
item_based_recommender.get_recommendations_info().head(50)

,track_id,name,artist,energy
0,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
1,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.4700
2,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
3,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
4,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
5,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
6,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
7,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
8,TRSBMHN128F426BF7E,Innocent Son,Fleet Foxes,0.0201
9,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.9670


In [120]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
8610,TRBIVWU128F92CA9D2,Skin and Bones,Foo Fighters,0.396


In [121]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
1272,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.47


In [122]:
recommended_song = item_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
5152,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.497


In [123]:
item_based_recommender.get_recommendations_info().head(50)

,track_id,name,artist,energy
0,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
1,TRKOHVA128F426BF70,Heard Them Stirring,Fleet Foxes,0.4700
2,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
3,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
4,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
5,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
6,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
7,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
8,TRSBMHN128F426BF7E,Innocent Son,Fleet Foxes,0.0201
9,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.9670


We will compare the results with weighted and binary ratings

In [185]:
def compare_lists(list1, list2):
    min_len = min(len(list1), len(list2))

    for i in range(len(list1)):
        if list1[i] != list2[i]:
            print(f"They defer in position {i}")
            break
    else:
        print("They are the same for all positions up to the minimum length.")


#Random subset of users
random_generator = np.random.default_rng(seed=RANDOM_STATE)
random_users = random_generator.choice(np.arange(len(user_uniques)), size=20, replace=False)

first_different_recommendation = []

for rand_user in random_users:
    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = item_based_recommender.get_recommendations_ids()

    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = item_based_recommender.get_recommendations_ids()

    for i in range(len(weighted_recommendations)):
        if weighted_recommendations[i] != binary_recommendations[i]:
            first_different_recommendation.append(i)
            break
    else:
        first_different_recommendation.append(-1)

print("First different recommendation for each user: (-1 = every recommendation is the same)")
print(first_different_recommendation)

First different recommendation for each user: (-1 = every recommendation is the same)
[0, 0, 0, 0, 2, 8, -1, 0, 3, 11, 0, 0, 0, 0, 0, 1, 2, 2, 3, 8]


We can see that recommendations defer in most of the cases. We will analyse a case where the recommendations are not the same

In [178]:
analized_user = random_users[0]

In [179]:
item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(analized_user, n=200)
weighted_recommendations = item_based_recommender.get_recommendations_info()

item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
item_based_recommender.make_recommendations(analized_user, n=200)
binary_recommendations = item_based_recommender.get_recommendations_info()

In [180]:
weighted_recommendations.head(20)

,track_id,name,artist,energy
0,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
1,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
2,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
3,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633
4,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
5,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
6,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
7,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
8,TRIQIAF128E0788692,Try a Little Tenderness,Michael Bublé,0.245
9,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508


In [181]:
binary_recommendations.head(20)

,track_id,name,artist,energy
0,TRVSJOM12903CD2DC1,One Less Lonely Girl,Justin Bieber,0.750
1,TRDGUPO128F9332CF6,Down To Earth,Bungle,0.904
2,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,0.790
3,TRZEDRT12903CD2DCC,Runaway Love,Justin Bieber,0.633
4,TRCKWGF12903CD2DCD,Never Let You Go,Third Eye Blind,0.940
5,TRIQIAF128E0788692,Try a Little Tenderness,Michael Bublé,0.245
6,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
7,TRTKLFX12903CD2DC2,First Dance,Justin Bieber,0.520
8,TRZYAGJ128F9332CEF,Favorite Girl,Justin Bieber,0.681
9,TRLNFKN128F931BAF2,The Tip Of The Iceberg,Owl City,0.772


We can see that even the order is not the same, most of the items are. Now we will calculate how many of the items recomended with the weighted ratings are recommended using the binary ratings

In [182]:
weighted_ids = set(weighted_recommendations['track_id'].head(50))
binary_ids = set(binary_recommendations['track_id'].head(50))
porcentaje = len(weighted_ids & binary_ids) / len(weighted_ids) * 100
print(f"Percentage of wieghted recommendations that are in binary recomendations: {porcentaje:.2f}%")

Percentage of wieghted recommendations that are in binary recomendations: 96.00%


84% of the items are the same in the first 50 recommendations of both systems. So, for this user, the use of weighted or binary rating system does not seem to have much impact. We will study the overlap of the other users of the subset

In [183]:
percentage_overlap_20 = []
percentage_overlap_50 = []
percentage_overlap_200 = []

for rand_user in random_users:
    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_weighted, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    weighted_recommendations = item_based_recommender.get_recommendations_info()

    item_based_recommender = ItemBasedRecommender(interaction_matrix_csc_binary, item_based_similarity, track_uniques, df_music_info)
    item_based_recommender.make_recommendations(rand_user, n=200)
    binary_recommendations = item_based_recommender.get_recommendations_info()

    weighted_ids_20 = set(weighted_recommendations['track_id'].head(20))
    binary_ids_20 = set(binary_recommendations['track_id'].head(20))

    weighted_ids_50 = set(weighted_recommendations['track_id'].head(50))
    binary_ids_50 = set(binary_recommendations['track_id'].head(50))

    weighted_ids_200 = set(weighted_recommendations['track_id'])
    binary_ids_200 = set(binary_recommendations['track_id'])


    overlap = round(len(weighted_ids_20 & binary_ids_20) / len(weighted_ids_20) * 100, 2)
    percentage_overlap_20.append(overlap)

    overlap = round(len(weighted_ids_50 & binary_ids_50) / len(weighted_ids_50) * 100, 2)
    percentage_overlap_50.append(overlap)

    overlap = round(len(weighted_ids_200 & binary_ids_200) / len(weighted_ids_200) * 100, 2)
    percentage_overlap_200.append(overlap)

print("Percentage of overlap between weighted and binary recommendations for each user:")
print(f"First 20 recommendations: {percentage_overlap_20}")
print(f"First 50 recommendations: {percentage_overlap_50}")
print(f"First 200 recommendations: {percentage_overlap_200}")

print(f"Average percentage of overlap for 20 first recommendations: {np.mean(percentage_overlap_20):.2f}%")
print(f"Median percentage of overlap for 20 first recommendations: {np.median(percentage_overlap_20):.2f}%")

print(f"Average percentage of overlap for 50 first recommendations: {np.mean(percentage_overlap_50):.2f}%")
print(f"Median percentage of overlap for 50 first recommendations: {np.median(percentage_overlap_50):.2f}%")

print(f"Average percentage of overlap for 200 first recommendations: {np.mean(percentage_overlap_200):.2f}%")
print(f"Median percentage of overlap for 200 first recommendations: {np.median(percentage_overlap_200):.2f}%")

Percentage of overlap between weighted and binary recommendations for each user:
First 20 recommendations: [95.0, 50.0, 70.0, 25.0, 65.0, 100.0, 100.0, 40.0, 60.0, 95.0, 85.0, 60.0, 65.0, 45.0, 40.0, 70.0, 95.0, 75.0, 95.0, 100.0]
First 50 recommendations: [96.0, 58.0, 72.0, 72.0, 94.0, 88.0, 100.0, 66.0, 74.0, 100.0, 58.0, 82.0, 80.0, 86.0, 72.0, 74.0, 78.0, 96.0, 86.0, 86.0]
First 200 recommendations: [95.5, 69.0, 80.0, 78.5, 89.0, 90.5, 100.0, 73.5, 93.0, 97.5, 85.5, 72.5, 78.0, 88.0, 55.5, 80.5, 75.0, 95.0, 80.5, 87.5]
Average percentage of overlap for 20 first recommendations: 71.50%
Median percentage of overlap for 20 first recommendations: 70.00%
Average percentage of overlap for 50 first recommendations: 80.90%
Median percentage of overlap for 50 first recommendations: 81.00%
Average percentage of overlap for 200 first recommendations: 83.22%
Median percentage of overlap for 200 first recommendations: 83.00%


We can see that there are not much differences in most of the cases using the weighted or binary rating method

## User based collaborative filtering

In [ ]:
#user_similarity = cosine_similarity(interaction_matrix_csr_weighted, dense_output=False)

We cant perform that operation because the amount of RAM needed, so we can not precalculate the similarities

In [ ]:
class UserBasedRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, num_neighbors=10):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.num_neighbors = num_neighbors
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=5):
        self.user_index = user_index

        user_ratings = self.interaction_matrix[user_index]

        user_similarities = cosine_similarity(user_ratings, self.interaction_matrix).flatten()

        user_similarities[user_index] = -np.inf

        top_n_neighbors_index = np.argpartition(user_similarities, -self.num_neighbors)[-self.num_neighbors:]
        top_n_neighbors_index = top_n_neighbors_index[np.argsort(user_similarities[top_n_neighbors_index])[::-1]]

        #print(top_n_neighbors_index)

        neighbors_mean_ratings = self.interaction_matrix[top_n_neighbors_index].mean(axis=0).A1

        user_ratings = self.interaction_matrix[user_index]
        interacted = user_ratings.nonzero()[1]
        neighbors_mean_ratings[interacted] = -np.inf #Asign -np.inf to already listened songs by the user

        top_n_index = np.argpartition(neighbors_mean_ratings, -n)[-n:]
        top_n_index = top_n_index[np.argsort(neighbors_mean_ratings[top_n_index])[::-1]]

        rating_list = []
        for index in top_n_index:
            rating_list.append(float(neighbors_mean_ratings[index]))

        rating_array = np.array(rating_list)
        
        # print('Num of elements of rating_list > 0:', np.sum(rating_array > 0))
        # print('Num of elements of rating_list = 0:', np.sum(rating_array == 0))

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Mean Rating: {neighbors_mean_ratings[index]}")
        
        track_ids = self.track_uniques[top_n_index].tolist()

        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()


        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids_ordered = [track_id for track_id, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

### Example

In [ ]:
user_index = 0 #User for wich recommendations will be made

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [ ]:
user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
user_based_recommender.make_recommendations(user_index, n=20)

In [ ]:
# user_listened_songs = df_users[df_users['user_id'] == user_uniques[201670]].track_id
# df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

In [ ]:
user_based_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.967
1,TRPGPDK12903CCC651,Bring Me To Life,Katherine Jenkins,0.563
2,TRJVHHB128F423786F,Aluga-se-vende,Móveis Coloniais de Acaju,0.799
3,TRYFJBZ128E0783C26,Two Way Monologue,Sondre Lerche,0.735
4,TRYHRDD128E0781847,Counter Spark,Sondre Lerche,0.648
5,TRFVBZA128F423787B,Do mesmo ar,Móveis Coloniais de Acaju,0.802
6,TRXWLLE128F92FC125,Hope You're Feeling Better,Santana,0.662
7,TRUKVPU128F147B59B,Love Vibration,Josh Rouse,0.663
8,TRBQCKI128F92E2DE5,Graveyard Disciples,Black Label Society,0.796
9,TRMNLOT128F1473B10,I Ain't Marching Anymore,Phil Ochs,0.492


In [ ]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
34798,TRDQHPF12903CDA54E,Open All Night,Bruce Springsteen,0.386


In [ ]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
30602,TRMKOST128E079939B,Immigrant Workforce,Bedouin Soundclash,0.47


In [ ]:
recommended_song = user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
32790,TRMNLOT128F1473B10,I Ain't Marching Anymore,Phil Ochs,0.492


In [ ]:
user_based_recommender.get_recommendations_info().head(10)

,track_id,name,artist,energy
0,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.967
1,TRPGPDK12903CCC651,Bring Me To Life,Katherine Jenkins,0.563
2,TRJVHHB128F423786F,Aluga-se-vende,Móveis Coloniais de Acaju,0.799
3,TRYFJBZ128E0783C26,Two Way Monologue,Sondre Lerche,0.735
4,TRYHRDD128E0781847,Counter Spark,Sondre Lerche,0.648
5,TRFVBZA128F423787B,Do mesmo ar,Móveis Coloniais de Acaju,0.802
6,TRXWLLE128F92FC125,Hope You're Feeling Better,Santana,0.662
7,TRUKVPU128F147B59B,Love Vibration,Josh Rouse,0.663
8,TRBQCKI128F92E2DE5,Graveyard Disciples,Black Label Society,0.796
9,TRMNLOT128F1473B10,I Ain't Marching Anymore,Phil Ochs,0.492


If we take the number of total users as neighbor count the system will recommend the songs that the user has not listed to in order of total play counts of all the users

In [ ]:
user_index = 7 #User for wich recommendations will be made
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
59,TRPFYYL128F92F7144,Float On,Modest Mouse,0.893
1823,TRWIKPM128F931F1D2,If It Means a Lot to You,A Day to Remember,0.536
1962,TRFXWSD128F93173BF,Gold Guns Girls,Metric,0.357
2485,TRLVQME128F931BAF3,Vanilla Twilight,Owl City,0.523
5147,TRXUWEC128F426BE3F,Skinny Love,Bon Iver,0.257
6929,TRGMZNT128F92DE267,Tim McGraw,Taylor Swift,0.445
12784,TRPUXCZ12903C99C90,Head Full Of Doubt/Road Full Of Promise,The Avett Brothers,0.472
19724,TROMKCG128F9320C09,Uprising,Sabaton,0.866


In [ ]:
user_based_recommender = UserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=464573)
user_based_recommender.make_recommendations(user_index, n=50)

In [ ]:
user_based_recommender.get_recommendations_info().head(10)

,track_id,name,artist,energy
0,TRONYHY128F92C9D11,Revelry,Kings of Leon,0.748
1,TRUFTBY128F93450B8,Alejandro,Lady Gaga,0.800
2,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.572
3,TRGCHLH12903CB7352,Party In The U.S.A.,The Barden Bellas,0.360
4,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,0.907
5,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508
6,TRPGPDK12903CCC651,Bring Me To Life,Katherine Jenkins,0.563
7,TRWAQOC12903CB84CA,Sexy Bitch,David Guetta,0.595
8,TRSUSWW128F93463BF,Imma Be,Black Eyed Peas,0.539
9,TRCRCBT128F4260DD1,Such Great Heights,The Postal Service,0.814


In [ ]:
num_distinct_users = df_users['user_id'].nunique()
track_playcounts = df_users.groupby('track_id')['playcount'].sum().sort_values(ascending=False)
top_5_track_ids = track_playcounts.index[:5]
top_5_df = track_playcounts.loc[top_5_track_ids].reset_index()
top_5_df.columns = ['track_id', 'total_playcount']
top_5_df.merge(df_music_info, on='track_id')

,track_id,total_playcount,name,artist,energy
0,TRONYHY128F92C9D11,274333,Revelry,Kings of Leon,0.748
1,TRUFTBY128F93450B8,76890,Alejandro,Lady Gaga,0.800
2,TRZNAHL128F9327D5A,71495,Gears,Miss May I,0.975
3,TRPFYYL128F92F7144,69767,Float On,Modest Mouse,0.893
4,TRXWAZC128F9314B3E,66304,Heartbreak Warfare,John Mayer,0.572


The recommendation on the top 

### KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:

class KnnUserBasedRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, num_neighbors=10):
        self.interaction_matrix = interaction_matrix
        self.knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=num_neighbors, n_jobs=-1)
        self.knn_model.fit(interaction_matrix)
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info
        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, has been recommended)

    def make_recommendations(self, user_index, n=5):
        self.user_index = user_index

        _, neighbors_indices = self.knn_model.kneighbors(self.interaction_matrix[user_index])

        #print(neighbors_indices)

        neighbors_interaction_matrix = self.interaction_matrix[neighbors_indices.flatten()]

        neighbors_mean_ratings = neighbors_interaction_matrix.mean(axis=0).A1 # Converts to 1D array

        user_ratings = self.interaction_matrix[user_index]
        interacted = user_ratings.nonzero()[1]
        neighbors_mean_ratings[interacted] = -np.inf #Asign -np.inf to already listened songs by the user

        top_n_index = np.argpartition(neighbors_mean_ratings, -n)[-n:]
        top_n_index = top_n_index[np.argsort(neighbors_mean_ratings[top_n_index])[::-1]]

        # for index in top_n_index:
        #     print(f"Track ID: {self.track_uniques[index]}, Mean Rating: {neighbors_mean_ratings[index]}")
        
        track_ids = self.track_uniques[top_n_index].tolist()

        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        # print(df_filtered)

        self.recommendations = [(track_id, energy, False) for track_id, energy in df_filtered.itertuples(index=False, name=None)]

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _ = self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _ in self.recommendations]
    
    
    def get_recommendations_info(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        track_ids_ordered = [track_id for track_id, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids_ordered].reset_index()
        return df_ordered

In [ ]:
user_index = 0 #User for wich recommendations will be made

In [ ]:
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [ ]:
knn_user_based_recommender = KnnUserBasedRecommender(interaction_matrix_csr_binary, track_uniques, df_music_info, num_neighbors=10)
knn_user_based_recommender.make_recommendations(user_index, n=5)

In [ ]:
user_based_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRONYHY128F92C9D11,Revelry,Kings of Leon,0.748
1,TRUFTBY128F93450B8,Alejandro,Lady Gaga,0.800
2,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.572
3,TRGCHLH12903CB7352,Party In The U.S.A.,The Barden Bellas,0.360
4,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,0.907
5,TRCPXID128F92D5D3C,Halo,Depeche Mode,0.508
6,TRPGPDK12903CCC651,Bring Me To Life,Katherine Jenkins,0.563
7,TRWAQOC12903CB84CA,Sexy Bitch,David Guetta,0.595
8,TRSUSWW128F93463BF,Imma Be,Black Eyed Peas,0.539
9,TRCRCBT128F4260DD1,Such Great Heights,The Postal Service,0.814


In [ ]:
recommended_song = knn_user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
26300,TRYHRDD128E0781847,Counter Spark,Sondre Lerche,0.648


In [ ]:
recommended_song = knn_user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
32074,TRXWLLE128F92FC125,Hope You're Feeling Better,Santana,0.662


In [ ]:
recommended_song = knn_user_based_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
24941,TRYFJBZ128E0783C26,Two Way Monologue,Sondre Lerche,0.735


In [ ]:
knn_user_based_recommender.get_recommendations_info().head(20)

,track_id,name,artist,energy
0,TRVCFUI128F92E1D42,Live-In Skin,Foo Fighters,0.967
1,TRYHRDD128E0781847,Counter Spark,Sondre Lerche,0.648
2,TRGYKTN12903CED295,I Always Get What I Want,Avril Lavigne,0.914
3,TRYFJBZ128E0783C26,Two Way Monologue,Sondre Lerche,0.735
4,TRXWLLE128F92FC125,Hope You're Feeling Better,Santana,0.662


## Matrix Factorization: Alternating Least Squares (ALS)

Aleternating Least Squares performs very well for sparse and implicit data. So we shall calculate the interaction matrix as the playcount or as proposed in the original ALS paper:

$$
\text{confidence}_{ui} = 1 + \alpha \cdot r_{ui}
$$

TODO implementar eso

In [ ]:
interaction_matrix_user_item = csr_matrix((df_users_rating['playcount'], (user_codes, track_codes)), shape=(len(user_uniques), len(track_uniques)))


In [ ]:
from implicit.als import AlternatingLeastSquares

In [ ]:
class ALSRecommender:
    def __init__(self, interaction_matrix, track_uniques, df_music_info, als_model=None):
        self.interaction_matrix = interaction_matrix
        self.track_uniques = track_uniques
        self.df_music_info = df_music_info

        if als_model is None:
            self.als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
            self.als_model.fit(self.interaction_matrix)
        else:
            self.als_model = als_model

        self.user_index = None
        self.recommendations = None # List of tuples (track_id, energy, similarity, has been recommended)

    def make_recommendations(self, user_index, n=100):
        self.user_index = user_index

        user_items = self.interaction_matrix.tocsr()[user_index]


        top_n_recommendations_indexes, top_n_recommendations_scores = self.als_model.recommend(user_index, user_items, N=n, filter_already_liked_items=True)


        track_ids = self.track_uniques[top_n_recommendations_indexes].tolist()
        
        df_filtered = self.df_music_info.set_index('track_id').loc[track_ids][['energy']].reset_index()

        self.recommendations = [(track_id, energy, similarity, False) for (track_id, energy), similarity in zip(df_filtered.itertuples(index=False, name=None), top_n_recommendations_scores)]
        return self.recommendations

    
    def recommend_song(self, energy, energy_margin=0.05):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        
        closest_track_index = None
        distance_to_energy = float('inf')

        for i, (track_id, track_energy, similarity, has_been_recommended) in enumerate(self.recommendations):
            distance = abs(track_energy - energy)

            if not has_been_recommended and distance <= energy_margin:
                self.recommendations[i] = (track_id, track_energy, similarity, True)
                return (track_id, track_energy)
            
            if not has_been_recommended and distance < distance_to_energy:
                closest_track_index = i
                distance_to_energy = distance
        
        if closest_track_index is not None:
            track_id, track_energy, _, _= self.recommendations[closest_track_index]
            self.recommendations[closest_track_index] = (track_id, track_energy, similarity, True)
            return (track_id, track_energy)

        raise ValueError("All recommendations have already been recommended")


    def get_recommendations(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return self.recommendations


    def get_recommendations_ids(self):
        if self.recommendations is None:
            raise ValueError("No recommendations available. Please call make_recommendations first.")
        return [track_id for track_id, _, _, _ in self.recommendations]
    
    def get_recommendations_info(self):
        track_ids = [track_id for track_id, _, _, _ in self.recommendations]
        df_ordered = self.df_music_info.set_index('track_id').loc[track_ids].reset_index()
        return df_ordered

### Example

In [ ]:
user_index = 0 #User for wich recommendations will be made
user_listened_songs = df_users[df_users['user_id'] == user_uniques[user_index]].track_id
df_music_info[df_music_info['track_id'].isin(user_listened_songs)]

,track_id,name,artist,energy
79,TRAAHSY128F147BB5C,Speed of Sound,Coldplay,0.8980
796,TRDLMWP128F426BF6C,Ragged Wood,Fleet Foxes,0.6850
1158,TRPUGUW128F426BF6F,He Doesn't Know Why,Fleet Foxes,0.5580
1743,TRRYCBO128F932A2C7,Love Shack,The B-52's,0.9020
2350,TRFUPBA128F934F7E1,Paper Gangsta,Lady Gaga,0.5300
2585,TRVODUZ128F934D094,All That We Perceive,Thievery Corporation,0.8340
2722,TRHDDQG12903CB53EE,Heaven's Gonna Burn Your Eyes,Thievery Corporation,0.3370
5416,TRTUCUY128F92E1D24,Stacked Actors,Foo Fighters,0.9340
5428,TRLXSNR128F429361D,Generator,Foo Fighters,0.9240
5819,TRADVZX128F426BF79,Sun Giant,Fleet Foxes,0.0626


In [ ]:
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20, num_threads=-1, random_state=RANDOM_STATE)
als_model.fit(interaction_matrix_user_item)

c:\Users\Dani\.conda\envs\Master-IA\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
als_recommender = ALSRecommender(interaction_matrix_user_item, track_uniques, df_music_info, als_model=als_model)
als_recommender.make_recommendations(0, n=20)
als_recommender.get_recommendations_info()

,track_id,name,artist,energy
0,TRUJOHU128F424E6A6,White Winter Hymnal,Fleet Foxes,0.4970
1,TRQEMRN128F933B4B2,Sun It Rises,Fleet Foxes,0.2750
2,TRGEIDA128F933B4B8,Tiger Mountain Peasant Song,Fleet Foxes,0.1460
3,TRVRIBZ128F426BF71,Your Protector,Fleet Foxes,0.5370
4,TRLXFKL128F426BF75,Oliver James,Fleet Foxes,0.0635
5,TRKABMK128F426BF72,Meadowlarks,Fleet Foxes,0.1660
6,TRFTUIW128E0784B9F,Bubble Toes,Jack Johnson,0.5320
7,TRDJRES128F933B4BA,Quiet Houses,Fleet Foxes,0.6680
8,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,0.5720
9,TRCQYJE128E078FA49,The News,Jack Johnson,0.0818


In [ ]:
recommended_song = als_recommender.recommend_song(0.4)
df_music_info[df_music_info['track_id'] == recommended_song[0]]

,track_id,name,artist,energy
16769,TREQOIZ128F42BCE96,I'm Gonna Find Another You,John Mayer,0.342


## Hybrid system

In [ ]:
with open('data.txt', 'r') as file:
    data_array = [line.strip() for line in file]

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'